# Import dependencies

1. Descargamos las librerías necesarias para la realización del script:

In [1]:
################  TRANSFORMERS ################
!pip  install transformers --quiet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import Trainer
!pip install transformers[torch] --quiet
!pip install accelerate -U --quiet
############## OPTUNA
! pip install optuna
import optuna
from optuna import Trial
from typing import Dict, Union
import threading
! pip install optuna-dashboard
! optuna-dashboard sqlite:///db.sqlite3
################  DATASETS ################
!pip install Datasets --quiet
from datasets import DatasetDict, Dataset
# Librería para montar drive en GoogleColab y descargar el dataset que vamos a usar:
from google.colab import drive  #Montador de drive


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 1967, in _exec_single_context
    self.dialect.do_execute(
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 924, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: version_info

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 72, in _create_scoped_session
    yield session
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 1042, in _init_version_info_model
    version_info = models.VersionInfoModel.find(session)
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/models.py", line 578, in find
    version_info = session.query(cls).one_or_none()
  File "/usr/local/lib/python3.10/dist-packages/sqlalche

In [2]:
# Importamos otros modulos que necesitaremos para nuestra aproximación:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
################  MÉTRICAS ################
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Librería usada para la división del conjunto de datos en 3 conjuntos (Entrenamiento, Validación y Test)
from sklearn.model_selection import train_test_split
import gc

# 2. Importamos el dataset

In [3]:
# 1.Obtenemos de drive el dataframe con el que vamos a trabajar
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/datasets/01 Exist2021_Esp.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df.head()

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,es,Nadie te va a tratar tan bien como un hombre q...,sexist,sexual-violence
1,EXIST2021,2,twitter,es,"@lindagisela74 Que rica putita obediente, afor...",sexist,stereotyping-dominance
2,EXIST2021,3,twitter,es,@BicireporteraDF Yo lo hice a los 18 años por ...,non-sexist,non-sexist
3,EXIST2021,4,twitter,es,las cosas q sueño son indicios de que yo enrea...,non-sexist,non-sexist
4,EXIST2021,5,twitter,es,"Pero a la niña le gustó desde que lo vió, así ...",non-sexist,non-sexist


In [5]:
# 2. Cambiamos el nombre de las columnas e eliminamos aquellas que no necesitamos:
columns_to_remove = ['test_case', 'id', 'source','language','task1']
df = df.rename(columns = {"task2": "label"}).drop(columns=columns_to_remove, axis=1)
df.head()

,text,label
0,Nadie te va a tratar tan bien como un hombre q...,sexual-violence
1,"@lindagisela74 Que rica putita obediente, afor...",stereotyping-dominance
2,@BicireporteraDF Yo lo hice a los 18 años por ...,non-sexist
3,las cosas q sueño son indicios de que yo enrea...,non-sexist
4,"Pero a la niña le gustó desde que lo vió, así ...",non-sexist


In [6]:
#Cambiamos los valores nominales de sexista y no sexista a valores numéricos
df['label'] = df['label'].replace(['non-sexist','sexual-violence', 'stereotyping-dominance',
 'misogyny-non-sexual-violence' ,'ideological-inequality', 'objectification'],[0,1,2,3,4,5])

In [7]:
#4. Eliminamos cualquier fila que haya podido quedar en blanco:
df = df.dropna()

In [8]:
df.head()

,text,label
0,Nadie te va a tratar tan bien como un hombre q...,1
1,"@lindagisela74 Que rica putita obediente, afor...",2
2,@BicireporteraDF Yo lo hice a los 18 años por ...,0
3,las cosas q sueño son indicios de que yo enrea...,0
4,"Pero a la niña le gustó desde que lo vió, así ...",0


https://www.youtube.com/watch?v=ZvsH09XGuZ0

### 2.1 - *Separamos el dataset en varios conjuntos (Train, Validación y Test)*

In [9]:
# Dividiremos el dataset en el 80% para el entrenamiento, el 10% para el proceso de validation, y un 10% para testear los resultados del modelo.
train_df, valtest_df = train_test_split(df, test_size = 0.2, random_state = 42)
val_df, test_df = train_test_split(valtest_df, test_size = 0.5, random_state = 42)
train_df.shape, val_df.shape, test_df.shape

((9063, 2), (1133, 2), (1133, 2))

In [10]:
train = Dataset.from_pandas(train_df)
validation = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(test_df)

In [11]:
#Lo convertimos en un solo dataset:

TWS = DatasetDict({
  'train': train,
  'val': validation,
  'test': test})
# Dataset.from_pandas genera una columna denominada index_0 qeu debemos eliminar:

TWS = TWS.remove_columns(["__index_level_0__"])

In [12]:
#Especificamos el modelo a adaptar a nuestro problema:
model_ckpt = 'Twitter/twhin-bert-base'
# y descargamos su tokenizador:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
#Debemos definir una función para tokenizar el dataset y que sea gestionable por nuestro modelo:
def tokenizador(batch):
  return tokenizer( batch["text"], padding = True,  max_length=128, truncation = True, return_tensors="pt")

In [14]:
# Para poder aplicarlo al todo el diccionario utilizaremos la función map:
TWS_Encoded = TWS.map(tokenizador, batched = True,batch_size = None )

Map:   0%|          | 0/9063 [00:00<?, ? examples/s]

Map:   0%|          | 0/1133 [00:00<?, ? examples/s]

Map:   0%|          | 0/1133 [00:00<?, ? examples/s]

In [15]:
#Necesitamos que los objetos sean de tipo torch
TWS_Encoded.set_format("torch", columns = ["label","input_ids", "attention_mask"])

In [16]:
#Debemos comprobar que la ejecucion se vaya a realizar sobre cuda:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# Definiremos un diccionario para poder traduccir las etiquetas o labels que nos devuelva nuestro modelo:
Etiqueta_dato = {}
Dato_Etiqueta = {}

etiquetas = ['non-sexist','sexual-violence', 'stereotyping-dominance',
 'misogyny-non-sexual-violence' ,'ideological-inequality', 'objectification']
for dato, nombre_etiqueta in enumerate(etiquetas):
  Dato_Etiqueta[nombre_etiqueta] = str(dato)
  Etiqueta_dato[str(dato)]= nombre_etiqueta


Etiqueta_dato , Dato_Etiqueta

({'0': 'non-sexist',
  '1': 'sexual-violence',
  '2': 'stereotyping-dominance',
  '3': 'misogyny-non-sexual-violence',
  '4': 'ideological-inequality',
  '5': 'objectification'},
 {'non-sexist': '0',
  'sexual-violence': '1',
  'stereotyping-dominance': '2',
  'misogyny-non-sexual-violence': '3',
  'ideological-inequality': '4',
  'objectification': '5'})

# 3 - Análisis de Hiperparametros:

In [18]:
#Debemos comprobar que la ejecucion se vaya a realizar sobre cuda:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
import optuna
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

# Assume you have a train_dataset and a eval_dataset
# train_dataset = ...
# eval_dataset = ...

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Compute the accuracy and F1 score
    accuracy, f1 = accuracy_score(labels, predictions), f1_score(labels, predictions, average='weighted')

    return {'accuracy': accuracy, 'f1': f1}

def objective(trial):
    model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = 6).to(device)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=trial.suggest_int('num_train_epochs', 1, 5),
        per_device_train_batch_size= trial.suggest_categorical("batch_size", [8, 12, 16, 32]),
        per_device_eval_batch_size= trial.suggest_categorical("batch_size", [8, 12, 16, 32]),
        learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        weight_decay=trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
        evaluation_strategy='epoch',
        save_strategy='epoch',
        metric_for_best_model='accuracy',
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=TWS_Encoded['train'],
        eval_dataset=TWS_Encoded['val'],
        compute_metrics=compute_metrics,
    )

    trainer.train()

    metrics = trainer.evaluate()

    return metrics['eval_accuracy']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=8)

best_trial = study.best_trial

print(f"Best Trial: {best_trial.number}")
print(f"  Value: {best_trial.value}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2024-05-27 06:17:07,513] A new study created in memory with name: no-name-33b207b9-f602-45a5-9523-f5c5c14d6457
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-899ae2f2b9b2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
<ipython-input-19-899ae2f2b9b2>:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(...,

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,3.571000,2.550882,0.484554,0.316314
2,2.559300,2.010835,0.484554,0.316314
3,2.218700,1.732346,0.484554,0.316314
4,1.665700,1.536935,0.484554,0.316314


[I 2024-05-27 06:29:24,656] Trial 0 finished with value: 0.48455428067078554 and parameters: {'num_train_epochs': 4, 'batch_size': 12, 'learning_rate': 0.017558727741320897, 'weight_decay': 0.04110981058769919}. Best is trial 0 with value: 0.48455428067078554.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-899ae2f2b9b2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
<ipython-input-19-899ae2f2b9b2>:27: FutureWarning: suggest_loguniform has 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.129131,0.586937,0.551332
2,1.221800,0.958965,0.661077,0.650067


[I 2024-05-27 06:33:49,806] Trial 1 finished with value: 0.6610767872903796 and parameters: {'num_train_epochs': 2, 'batch_size': 32, 'learning_rate': 5.998011133104437e-05, 'weight_decay': 0.00722944119363268}. Best is trial 1 with value: 0.6610767872903796.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-899ae2f2b9b2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
<ipython-input-19-899ae2f2b9b2>:27: FutureWarning: suggest_loguniform has b

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.504600,1.540398,0.484554,0.316314
2,1.502300,1.534882,0.484554,0.316314
3,1.492700,1.526067,0.484554,0.316314
4,1.486700,1.524552,0.484554,0.316314
5,1.451800,1.521010,0.484554,0.316314


[I 2024-05-27 07:41:15,627] Trial 6 finished with value: 0.48455428067078554 and parameters: {'num_train_epochs': 5, 'batch_size': 8, 'learning_rate': 0.00033133917892571, 'weight_decay': 1.0265252266492303e-05}. Best is trial 1 with value: 0.6610767872903796.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-899ae2f2b9b2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
<ipython-input-19-899ae2f2b9b2>:27: FutureWarning: suggest_loguniform has 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.499500,1.486677,0.484554,0.316314


[I 2024-05-27 07:44:25,562] Trial 7 finished with value: 0.48455428067078554 and parameters: {'num_train_epochs': 1, 'batch_size': 12, 'learning_rate': 6.781338921439904e-05, 'weight_decay': 0.0008805806312682235}. Best is trial 1 with value: 0.6610767872903796.


Best Trial: 1
  Value: 0.6610767872903796
  Params: 
    num_train_epochs: 2
    batch_size: 32
    learning_rate: 5.998011133104437e-05
    weight_decay: 0.00722944119363268


In [27]:
from optuna.importance import get_param_importances

# Assume `study` is a completed Optuna study
param_importances = get_param_importances(study)

# Print the importances
for param, importance in param_importances.items():
    print(f"Parameter: {param}, Importance: {importance}")

Parameter: learning_rate, Importance: 0.5853658536585367
Parameter: batch_size, Importance: 0.3658536585365853
Parameter: num_train_epochs, Importance: 0.04878048780487801
Parameter: weight_decay, Importance: 0.0


In [29]:
param_importances

{'learning_rate': 0.5853658536585367,
 'batch_size': 0.3658536585365853,
 'num_train_epochs': 0.04878048780487801,
 'weight_decay': 0.0}

In [21]:
# 1. Definimos la función con los hyperparametros que queremos probar:
def my_hp_space(trial):
    return {
        "learning_rate" : trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
        "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16, 32]),
        #"per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8,12,16]),
        "num_train_epochs":trial.suggest_int("num_train_epochs", low=3, high=5)
        #"dropout_rate": trial.suggest_int("dropout_rate", low=0,1, high=0,4)
    }

In [22]:
# 2. Tenemos que definir una función de iniciado del modelo
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = 6)

In [23]:
# 3. vamos a necesitar para nuestro objeto trainer la función que calcula las métricas:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [24]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    per_device_eval_batch_size=64,
    evaluation_strategy = "epoch",
    logging_dir=None,                # directory for storing logs, set to None to disable
    logging_steps=5000,              # increase the number of steps between logs to reduce log size
    save_strategy="no"               # no saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
# 3. Crear un objeto entrenador con la función model_init, argumentos de entrenamiento, conjuntos de datos de entrenamiento y prueba, y función de evaluación:
trainer = Trainer(
    args=training_args,
    train_dataset = TWS_Encoded["train"],
    eval_dataset = TWS_Encoded["val"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Ejecutamos el estudio en optuna a través de la API dEL TRAINER.
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=my_hp_space,
    n_trials= 8,
)

[I 2024-05-27 07:44:26,836] A new study created in memory with name: no-name-e61df526-b9e2-463c-b750-e8d252fe67f4
<ipython-input-21-998a0ebb5a5e>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate" : trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.045155,0.608120,0.607315
2,No log,0.976512,0.668138,0.656686


[W 2024-05-27 07:51:44,421] Trial 0 failed with parameters: {'learning_rate': 4.3194403955107916e-05, 'batch_size': 16, 'num_train_epochs': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 210, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1885, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2216, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3238, in training_step
    loss = self.compute_loss(model, inputs)
  File "/usr/l

KeyboardInterrupt: 